# 0.0 IMPORTS

In [113]:
!pip install inflection
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 57.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=d4c3cb97b57480dd31ee50f3895c07a9e6b27d65e6c2f5256a208c3b22033211
  Stored in directory: /root/.cache/pip/wheels/2d/f0/a8/1094fca7a7e5d0d12ff56e0c64675d72aa5cc81a5fc200e849
Successfully bu

In [115]:
import pandas as pd
import numpy as np
import inflection
import seaborn as sns
import plotly.express as px
import streamlit as st
from google.colab import drive

## 0.1 Loading Data

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/Colab Notebooks/resultados_lvl/data/order_export_2023-04-17.csv'

In [5]:
df_raw = pd.read_csv(path, encoding='ISO-8859-1')

##0.2 Functions

In [6]:
def df_mes_selecionado(df,mes):
  df_mes = df.loc[df3['data'].dt.month == mes,:]
  return df_mes

# 1.0 DATA DESCRIPTION

In [7]:
df1 = df_raw.copy()

## 1.1 Rename Columns

In [8]:
df1.head()

,NOME,SOBRENOME,CONTA,DATA,HORA,DATAHORA,CODIGO,QUANTIDADE,PRECO,LADO,CORRETAGEM,BRUTO,TAXAS,LIQUIDO,DURACAO
0,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:03:27,2022-10-03 09:03:27,WINV22,12.0,114755.0,Compra,0.6,0.0,3.0,0.0,0
1,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:03:27,2022-10-03 09:03:27,WINV22,12.0,114805.0,Venda,0.6,120.0,3.0,112.8,0
2,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:03:58,2022-10-03 09:03:58,WINV22,12.0,114350.0,Venda,0.6,0.0,3.0,0.0,0
3,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:03:59,2022-10-03 09:03:59,WINV22,12.0,114300.0,Compra,0.6,120.0,3.0,112.8,1
4,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:04:22,2022-10-03 09:04:22,WINV22,12.0,114320.0,Venda,0.6,0.0,3.0,0.0,0


In [9]:
cols_new = list(map(lambda x: inflection.underscore(x), df1.columns))
df1.columns = cols_new

##1.2 Data Dimensions

In [10]:
df1.shape

(91984, 15)

##1.3 Data Types

In [11]:
df1.dtypes

nome           object
sobrenome      object
conta          object
data           object
hora           object
datahora       object
codigo         object
quantidade    float64
preco         float64
lado           object
corretagem    float64
bruto         float64
taxas         float64
liquido       float64
duracao         int64
dtype: object

In [12]:
df1['data'] = pd.to_datetime(df1['data'])
df1['datahora'] = pd.to_datetime(df1['datahora'])

##1.4 Check NA

In [13]:
df1.isna().sum()

nome          0
sobrenome     0
conta         0
data          0
hora          0
datahora      0
codigo        0
quantidade    0
preco         0
lado          0
corretagem    0
bruto         0
taxas         0
liquido       0
duracao       0
dtype: int64

# 2.0 VARIABLE FILTERING

In [14]:
df2 = df1.copy()

In [15]:
#Retirando operações que abrem a operação e pegando apenas o fechamento
df2 = df2.loc[(df2['liquido'] != 0.0) & (df2['bruto'] != 0.0),:]

In [16]:
#invertendo todos os valores de compra para venda e vice versa
df2['lado'] = df2['lado'].apply(lambda x: 'Compra' if x == 'Venda' else 'Venda')

In [17]:
#esse primeiro código gerou um warning
#df3_mes['valor_por_contrato'] = df3_mes['liquido']/df3_mes['quantidade']
df2 = df2.assign(valor_por_contrato = df2['liquido'] / df2['quantidade'])

In [18]:
df2 = df2.loc[df2['duracao'] < 32000,:]
df2.reset_index(drop=True)

,nome,sobrenome,conta,data,hora,datahora,codigo,quantidade,preco,lado,corretagem,bruto,taxas,liquido,duracao,valor_por_contrato
0,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:03:27,2022-10-03 09:03:27,WINV22,12.0,114805.0,Compra,0.60,120.0,3.0,112.8,0,9.40
1,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:03:59,2022-10-03 09:03:59,WINV22,12.0,114300.0,Venda,0.60,120.0,3.0,112.8,1,9.40
2,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:04:22,2022-10-03 09:04:22,WINV22,12.0,114270.0,Venda,0.60,120.0,3.0,112.8,0,9.40
3,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:04:31,2022-10-03 09:04:31,WINV22,12.0,114315.0,Compra,0.60,120.0,3.0,112.8,1,9.40
4,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:05:29,2022-10-03 09:05:29,WINV22,12.0,114375.0,Venda,0.60,120.0,3.0,112.8,1,9.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35501,Arielton,Silva,NF15,2023-04-14,17:57:29,2023-04-14 17:57:29,WDOK23,10.0,4920.5,Compra,0.50,100.0,8.2,82.6,137,8.26
35502,Felipe,Barcelos Vasconcellos,NF52,2023-04-14,17:58:30,2023-04-14 17:58:30,WDOK23,15.0,4920.0,Compra,0.75,150.0,12.3,123.9,30,8.26
35503,Felipe,Barcelos Vasconcellos,NF52,2023-04-14,17:59:36,2023-04-14 17:59:36,WDOK23,15.0,4918.0,Compra,0.75,-75.0,12.3,-101.1,10,-6.74
35504,Edilson,Jesus dos Santos Junior,NF47,2023-04-14,17:59:42,2023-04-14 17:59:42,WDOK23,15.0,4918.6,Compra,0.75,-60.0,12.3,-86.1,23,-5.74


In [19]:
df2['ranking'] = 0
df2.loc[df2['liquido']>0,'ranking'] = 1
df2.loc[df2['liquido']<0,'ranking'] = -1

In [20]:
df2['valor_por_acao'] = df2['liquido']/(df2['preco']*df2['quantidade'])

In [21]:
df2.head()

,nome,sobrenome,conta,data,hora,datahora,codigo,quantidade,preco,lado,corretagem,bruto,taxas,liquido,duracao,valor_por_contrato,ranking,valor_por_acao
1,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:03:27,2022-10-03 09:03:27,WINV22,12.0,114805.0,Compra,0.6,120.0,3.0,112.8,0,9.4,1,0.000082
3,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:03:59,2022-10-03 09:03:59,WINV22,12.0,114300.0,Venda,0.6,120.0,3.0,112.8,1,9.4,1,0.000082
5,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:04:22,2022-10-03 09:04:22,WINV22,12.0,114270.0,Venda,0.6,120.0,3.0,112.8,0,9.4,1,0.000082
7,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:04:31,2022-10-03 09:04:31,WINV22,12.0,114315.0,Compra,0.6,120.0,3.0,112.8,1,9.4,1,0.000082
9,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:05:29,2022-10-03 09:05:29,WINV22,12.0,114375.0,Venda,0.6,120.0,3.0,112.8,1,9.4,1,0.000082


# Visão Geral

In [22]:
df3 = df2.copy()

In [23]:
df3.head()

,nome,sobrenome,conta,data,hora,datahora,codigo,quantidade,preco,lado,corretagem,bruto,taxas,liquido,duracao,valor_por_contrato,ranking,valor_por_acao
1,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:03:27,2022-10-03 09:03:27,WINV22,12.0,114805.0,Compra,0.6,120.0,3.0,112.8,0,9.4,1,0.000082
3,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:03:59,2022-10-03 09:03:59,WINV22,12.0,114300.0,Venda,0.6,120.0,3.0,112.8,1,9.4,1,0.000082
5,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:04:22,2022-10-03 09:04:22,WINV22,12.0,114270.0,Venda,0.6,120.0,3.0,112.8,0,9.4,1,0.000082
7,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:04:31,2022-10-03 09:04:31,WINV22,12.0,114315.0,Compra,0.6,120.0,3.0,112.8,1,9.4,1,0.000082
9,Felipe,Barcelos Vasconcellos,NF52,2022-10-03,09:05:29,2022-10-03 09:05:29,WINV22,12.0,114375.0,Venda,0.6,120.0,3.0,112.8,1,9.4,1,0.000082


##1. Resultado total por estratégia no mês de abril

In [24]:
df3_mes = df_mes_selecionado(df3,4)  

In [25]:
df3_mes.head()

,nome,sobrenome,conta,data,hora,datahora,codigo,quantidade,preco,lado,corretagem,bruto,taxas,liquido,duracao,valor_por_contrato,ranking,valor_por_acao
87605,Felipe,Massaity Gerez,NF61,2023-04-03,09:00:59,2023-04-03 09:00:59,WDOK23,4.0,5073.50,Venda,0.200,200.0,3.280,193.040,7,48.26000,1,0.009512
87607,Felipe,Barcelos Vasconcellos,NF52,2023-04-03,09:01:04,2023-04-03 09:01:04,WDOK23,10.0,5074.00,Compra,0.500,300.0,8.200,282.600,3,28.26000,1,0.005570
87609,Luckas,Rhuan de Oliveira Ruiz Zanforlim,NF54,2023-04-03,09:01:29,2023-04-03 09:01:29,WDOK23,2.0,5072.00,Venda,0.100,-10.0,1.640,-13.480,4,-6.74000,-1,-0.001329
87611,Luckas,Rhuan de Oliveira Ruiz Zanforlim,NF54,2023-04-03,09:37:12,2023-04-03 09:37:12,WINJ23,3.0,102345.00,Compra,0.150,72.0,0.750,70.200,165,23.40000,1,0.000229
87616,Henrique,de França Santos,NF85,2023-04-03,10:03:32,2023-04-03 10:03:32,LWSA3,2500.0,4.85,Compra,0.606,75.0,2.728,68.351,2,0.02734,1,0.005637


###1.1 Abertura Futuros

In [26]:
#Todos os trades entre 09:00:00 e 09:10:00 dentro do mês
df3_hora_abertura_futuros = df3_mes.loc[(df3_mes['hora'] >= '09:00:00') & (df3_mes['hora'] <= '09:05:00'),:]

In [27]:
df3_hora_abertura_futuros.head()

,nome,sobrenome,conta,data,hora,datahora,codigo,quantidade,preco,lado,corretagem,bruto,taxas,liquido,duracao,valor_por_contrato,ranking,valor_por_acao
87605,Felipe,Massaity Gerez,NF61,2023-04-03,09:00:59,2023-04-03 09:00:59,WDOK23,4.0,5073.5,Venda,0.2,200.0,3.28,193.04,7,48.26,1,0.009512
87607,Felipe,Barcelos Vasconcellos,NF52,2023-04-03,09:01:04,2023-04-03 09:01:04,WDOK23,10.0,5074.0,Compra,0.5,300.0,8.20,282.60,3,28.26,1,0.005570
87609,Luckas,Rhuan de Oliveira Ruiz Zanforlim,NF54,2023-04-03,09:01:29,2023-04-03 09:01:29,WDOK23,2.0,5072.0,Venda,0.1,-10.0,1.64,-13.48,4,-6.74,-1,-0.001329
87908,Arielton,Silva,NF15,2023-04-04,09:00:43,2023-04-04 09:00:43,WDOK23,10.0,5072.5,Venda,0.5,300.0,8.20,282.60,3,28.26,1,0.005571
87910,Pedro,Augusto Inácio,NF35,2023-04-04,09:00:50,2023-04-04 09:00:50,WDOK23,6.0,5076.5,Compra,0.3,210.0,4.92,199.56,9,33.26,1,0.006552


####1.1.1 Total Sala

In [28]:
df3_abertura_futuros_total = df3_hora_abertura_futuros.loc[:,['liquido','codigo']].groupby(['codigo']).sum().reset_index()
px.bar(df3_abertura_futuros_total,x='codigo',y='liquido', labels={'codigo':'Ativo','liquido':'Resultado Líquido'})
df3_abertura_futuros_total

,codigo,liquido
0,DOLK23,2841.00
1,INDJ23,1168.60
2,WDOK23,789.43
3,WINJ23,-1037.00
4,WINM23,-113.40


####1.1.2 Total por operador

In [29]:
df3_abertura_futuros_por_op = df3_hora_abertura_futuros.loc[:,['liquido','nome','codigo']].groupby(['nome','codigo']).sum().reset_index()
df3_abertura_futuros_por_op

,nome,codigo,liquido
0,Arielton,DOLK23,1127.30
1,Arielton,WDOK23,378.20
2,Arielton,WINJ23,-1202.00
3,Arielton,WINM23,65.00
4,Edilson,DOLK23,1043.20
5,Edilson,INDJ23,1309.30
6,Eduardo Vinicius,DOLK23,-622.70
7,Eduardo Vinicius,WDOK23,232.60
8,Felipe,WDOK23,141.95
9,Felipe,DOLK23,84.10


####1.1.3 Média Operações Vencedoras



In [30]:
op_vencedoras = df3_mes['liquido'] > 0
df3_vencedoras_media = df3_hora_abertura_futuros.loc[op_vencedoras,['codigo','liquido']].groupby('codigo').agg({'liquido': ['mean','count','max']}).reset_index()
df3_vencedoras_media.columns = ['codigo','media_melhores','qtd_melhores','melhor']
df3_vencedoras_media

,codigo,media_melhores,qtd_melhores,melhor
0,DOLK23,505.975,8,1209.1
1,INDJ23,1309.300,1,1309.3
2,WDOK23,136.900,13,282.6
3,WINJ23,54.150,4,183.0
4,WINM23,65.000,1,65.0


####1.1.4 Média Operações Perdedoras

In [31]:
op_perdedoras = df3_mes['liquido'] < 0
df3_perdedoras_media = df3_hora_abertura_futuros.loc[op_perdedoras,['codigo','liquido']].groupby('codigo').agg({'liquido': ['mean','count','min']}).reset_index()
df3_perdedoras_media.columns = ['codigo','media_piores','qtd_piores','pior']
df3_perdedoras_media

,codigo,media_piores,qtd_piores,pior
0,DOLK23,-603.4000,2,-1040.9
1,INDJ23,-140.7000,1,-140.7
2,WDOK23,-82.5225,12,-267.4
3,WINJ23,-250.7200,5,-514.0
4,WINM23,-178.4000,1,-178.4


####1.1.5 Média de valor por pts

In [32]:
df3_media_p_contrato = df3_hora_abertura_futuros.loc[:,['valor_por_contrato','codigo']].groupby('codigo').mean().reset_index()
df3_media_p_contrato

,codigo,valor_por_contrato
0,DOLK23,56.8200
1,INDJ23,116.8600
2,WDOK23,-0.4444
3,WINJ23,-9.0000
4,WINM23,-19.0500


####1.1.6 Tempo Médio por Operação

In [33]:
df3_media_p_tempo = df3_hora_abertura_futuros.loc[:,['duracao','codigo']].groupby('codigo').mean().reset_index()
df3_media_p_tempo['duracao'] = round(df3_media_p_tempo['duracao'],2)
df3_media_p_tempo

,codigo,duracao
0,DOLK23,16.90
1,INDJ23,8.00
2,WDOK23,22.92
3,WINJ23,54.89
4,WINM23,100.00


####1.1.7 Tabela final

In [34]:
df3_combinado = pd.merge(df3_abertura_futuros_total,df3_vencedoras_media, on='codigo')
df3_combinado1 = pd.merge(df3_combinado, df3_perdedoras_media, on='codigo')
df3_combinado2 = pd.merge(df3_combinado1,df3_media_p_contrato, on='codigo')
df3_combinado3 = pd.merge(df3_combinado2,df3_media_p_tempo, on='codigo' )
nova_ordem_colunas = ['codigo','resultado','media_melhores','media_piores','melhor','pior','valor_por_contrato','qtd_melhores','qtd_piores','duracao']
df3_combinado3.reindex(columns=nova_ordem_colunas)
#Tabela Final
df3_combinado3['% vencedoras'] = round(df3_combinado3['qtd_melhores']/(df3_combinado3['qtd_melhores'] + df3_combinado3['qtd_piores']),2) 
df3_combinado3

,codigo,liquido,media_melhores,qtd_melhores,melhor,media_piores,qtd_piores,pior,valor_por_contrato,duracao,% vencedoras
0,DOLK23,2841.00,505.975,8,1209.1,-603.4000,2,-1040.9,56.8200,16.90,0.80
1,INDJ23,1168.60,1309.300,1,1309.3,-140.7000,1,-140.7,116.8600,8.00,0.50
2,WDOK23,789.43,136.900,13,282.6,-82.5225,12,-267.4,-0.4444,22.92,0.52
3,WINJ23,-1037.00,54.150,4,183.0,-250.7200,5,-514.0,-9.0000,54.89,0.44
4,WINM23,-113.40,65.000,1,65.0,-178.4000,1,-178.4,-19.0500,100.00,0.50


###1.2 Futuros Tarde

In [35]:
#Todos os trades após 09:05:00 e 17:00:00
hora_tarde_futuros = (df3_mes['hora'] > '09:05:00') & (df3_mes['hora'] <= '17:00:00')
ativo_futuros = df3_mes['codigo'].str.startswith(('WDO','WIN','DOL','IND')) 

####1.2.1 Total Sala

In [36]:
df3_futuros_tarde = df3_mes.loc[(hora_tarde_futuros) & (ativo_futuros),['liquido','codigo']].groupby(['codigo']).sum().reset_index()
df3_futuros_tarde['liquido'] = round(df3_futuros_tarde['liquido'],2)
#px.bar(df2_futuros_tarde,x='codigo',y='liquido', labels={'codigo':'Ativo','liquido':'Resultado Líquido'})
df3_futuros_tarde

,codigo,liquido
0,WDOK23,274.49
1,WINJ23,4663.00
2,WINM23,-414.80


####1.2.2 Total por Operador

In [37]:
df3_tarde_futuros_op =  df3_mes.loc[(hora_tarde_futuros) & (ativo_futuros),['liquido','nome','codigo']].groupby(['nome','codigo']).sum().reset_index()
df3_tarde_futuros_op

,nome,codigo,liquido
0,Arielton,WDOK23,107.60
1,Arielton,WINJ23,1544.20
2,Edilson,WINJ23,271.40
3,Edilson,WINM23,-8.00
4,Eduardo Vinicius,WDOK23,1157.60
5,Eduardo Vinicius,WINJ23,1381.00
6,Felipe,WDOK23,-572.61
7,Felipe,WINJ23,1008.20
8,Felipe,WINJ23,-92.00
9,Henrique,WINJ23,78.80


####1.2.3 Média Operações Vencedoras

In [38]:
op_vencedoras = df3_mes['liquido'] > 0
df3_vencedoras_media_tarde = df3_mes.loc[(hora_tarde_futuros) & (ativo_futuros) & (op_vencedoras),['codigo','liquido']].groupby('codigo').agg({'liquido': ['mean','count','max']}).reset_index()
df3_vencedoras_media_tarde.columns = ['codigo','media_melhores','qtd_melhores','melhor']
df3_vencedoras_media_tarde

,codigo,media_melhores,qtd_melhores,melhor
0,WDOK23,94.392609,23,627.39
1,WINJ23,41.911811,200,684.00
2,WINM23,58.200000,1,58.20


####1.2.4 Média Op Perdedoras

In [39]:
op_perdedoras = df3_mes['liquido'] < 0
df3_perdedoras_media_tarde = df3_mes.loc[(hora_tarde_futuros) & (ativo_futuros) & (op_perdedoras),['codigo','liquido']].groupby('codigo').agg({'liquido': ['mean','count','min']}).reset_index()
df3_perdedoras_media_tarde.columns = ['codigo','media_piores','qtd_piores','pior']
df3_perdedoras_media_tarde

,codigo,media_piores,qtd_piores,pior
0,WDOK23,-63.218000,30,-572.61
1,WINJ23,-70.176644,53,-656.00
2,WINM23,-39.416667,12,-90.40


####1.2.5 Valor por contrato

In [40]:
df3_valor_p_contrato_tarde = df3_mes.loc[(hora_tarde_futuros) & (ativo_futuros),['codigo','valor_por_contrato']].groupby('codigo').mean().reset_index()
df3_valor_p_contrato_tarde

,codigo,valor_por_contrato
0,WDOK23,3.362547
1,WINJ23,2.621598
2,WINM23,-10.523077


####1.2.6 Tempo Operação

In [41]:
df3_tempo_op_tarde = df3_mes.loc[(hora_tarde_futuros) & (ativo_futuros),['codigo','duracao']].groupby('codigo').mean().reset_index()
df3_tempo_op_tarde

,codigo,duracao
0,WDOK23,161.849057
1,WINJ23,47.699605
2,WINM23,274.615385


####1.2.7 Tabela Final

In [42]:
df3_combinado = pd.merge(df3_futuros_tarde,df3_vencedoras_media_tarde, on='codigo')
df3_combinado1 = pd.merge(df3_combinado, df3_perdedoras_media_tarde, on='codigo')
df3_combinado2 = pd.merge(df3_combinado1,df3_valor_p_contrato_tarde, on='codigo')
df3_combinado3 = pd.merge(df3_combinado2,df3_tempo_op_tarde, on='codigo' )
nova_ordem_colunas = ['codigo','resultado','media_melhores','media_piores','melhor','pior','valor_por_contrato','qtd_melhores','qtd_piores','duracao']
df3_combinado3.reindex(columns=nova_ordem_colunas)
#Tabela Final
df3_combinado3['% vencedoras'] = round(df3_combinado3['qtd_melhores']/(df3_combinado3['qtd_melhores'] + df3_combinado3['qtd_piores']),2) 
df3_combinado3

,codigo,liquido,media_melhores,qtd_melhores,melhor,media_piores,qtd_piores,pior,valor_por_contrato,duracao,% vencedoras
0,WDOK23,274.49,94.392609,23,627.39,-63.218000,30,-572.61,3.362547,161.849057,0.43
1,WINJ23,4663.00,41.911811,200,684.00,-70.176644,53,-656.00,2.621598,47.699605,0.79
2,WINM23,-414.80,58.200000,1,58.20,-39.416667,12,-90.40,-10.523077,274.615385,0.08


###1.3 Abertura Ações

In [43]:
df3_hora_abertura_acoes = df3_mes.loc[(df3_mes['hora'] >= '10:00:00') & (df3_mes['hora'] < '12:00:00'),:]
df3_abertura_acoes = df3_hora_abertura_acoes.loc[~df3_hora_abertura_acoes['codigo'].str.startswith(('WDO','WIN','DOL','IND','CCM')),:]

####1.3.1 Total Sala

In [ ]:
df3_total_ab_acoes = df3_abertura_acoes.loc[:,['liquido','codigo']].groupby('codigo').sum().reset_index().sort_values(by='liquido',ascending=False)
df3_total_ab_acoes['liquido'] = round(df3_total_ab_acoes['liquido'],2)
df3_total_ab_acoes

####1.3.2 Op Vencedoras

In [ ]:
op_vencedoras = df3_abertura_acoes['liquido'] > 0
df3_vencedoras_ab_acoes = df3_abertura_acoes.loc[op_vencedoras,['codigo','liquido']].groupby('codigo').agg({'liquido': ['mean','count','max']}).reset_index()
df3_vencedoras_ab_acoes.columns = ['codigo','media_melhores','qtd_melhores','melhor']
df3_vencedoras_ab_acoes['media_melhores'] = round(df3_vencedoras_ab_acoes['media_melhores'],2)
df3_vencedoras_ab_acoes['melhor'] = round(df3_vencedoras_ab_acoes['melhor'],2)
df3_vencedoras_ab_acoes

####1.3.3 Op Perdedoras

In [ ]:
op_perdedoras = df3_abertura_acoes['liquido'] < 0
df3_perdedoras_ab_acoes = df3_abertura_acoes.loc[op_perdedoras,['codigo','liquido']].groupby('codigo').agg({'liquido': ['mean','count','max']}).reset_index()
df3_perdedoras_ab_acoes.columns = ['codigo','media_piores','qtd_piores','pior']
df3_perdedoras_ab_acoes['media_piores'] = round(df3_perdedoras_ab_acoes['media_piores'],2)
df3_perdedoras_ab_acoes['pior'] = round(df3_perdedoras_ab_acoes['pior'],2)
df3_perdedoras_ab_acoes

####1.3.4 Valor por ação

In [ ]:
#Fazer valor por ação(LOTE X PREÇO)
df3_valor_p_ab_acao = df3_abertura_acoes.loc[:,['codigo','valor_por_acao']].groupby('codigo').mean().reset_index().sort_values(by='valor_por_acao',ascending=False)
df3_valor_p_ab_acao['valor_por_acao'] = df3_valor_p_ab_acao['valor_por_acao']*100
df3_valor_p_ab_acao

####1.3.5 Tempo por Operação

In [ ]:
df3_tempo_p_op_ab_acoes = df3_abertura_acoes.loc[:,['codigo','duracao']].groupby('codigo').mean().reset_index().sort_values(by='duracao',ascending=False)
df3_tempo_p_op_ab_acoes['duracao'] = round(df3_tempo_p_op_ab_acoes['duracao'],2)
df3_tempo_p_op_ab_acoes

####1.3.6 Tabela Final

In [55]:
df3_combinado = pd.merge(df3_total_ab_acoes,df3_vencedoras_ab_acoes, on='codigo')
df3_combinado1 = pd.merge(df3_combinado, df3_perdedoras_ab_acoes, on='codigo')
df3_combinado2 = pd.merge(df3_combinado1,df3_valor_p_ab_acao, on='codigo')
df3_combinado3 = pd.merge(df3_combinado2,df3_tempo_p_op_ab_acoes, on='codigo' )
nova_ordem_colunas = ['codigo','liquido','media_melhores','media_piores','melhor','pior','valor_por_acao','qtd_melhores','qtd_piores','duracao']
df3_combinado3.reindex(columns=nova_ordem_colunas)
df3_combinado3['% vencedoras'] = round(df3_combinado3['qtd_melhores']/(df3_combinado3['qtd_melhores'] + df3_combinado3['qtd_piores']),2) 
df3_combinado3

,codigo,liquido,media_melhores,qtd_melhores,melhor,media_piores,qtd_piores,pior,valor_por_acao,duracao,% vencedoras
0,CPLE3,1417.86,143.33,10,267.28,-15.45,1,-15.45,1.834750,194.91,0.91
1,BRFS3,1413.16,135.79,11,220.96,-80.52,1,-80.52,1.234187,37.92,0.92
2,PRIO3,1223.57,63.59,27,239.65,-41.12,12,-2.65,0.291767,184.69,0.69
3,EZTC3,1217.13,126.50,11,235.10,-58.11,3,-34.02,0.678144,109.57,0.79
4,RECV3,1061.32,138.83,8,238.10,-24.65,2,-23.48,0.776155,213.20,0.80
5,GOLL4,996.83,117.06,14,402.15,-58.36,11,-6.73,0.535255,172.40,0.56
6,USIM5,837.10,66.78,13,147.94,-15.53,2,-13.95,0.750062,182.60,0.87
7,AZUL4,825.03,89.32,11,186.83,-52.49,3,-26.96,0.537186,160.71,0.79
8,NTCO3,709.67,37.21,53,226.56,-50.50,25,-0.14,0.654622,88.33,0.68
9,MOVI3,693.28,126.66,6,191.36,-66.70,1,-66.70,1.548633,38.43,0.86


###1.4 Leilão Fechamento

In [71]:
df3_hora_leilao = df3_mes.loc[df3_mes['hora'] >= '15:00:00',:]
df3_hora_leilao = df3_hora_leilao.loc[~df3_hora_leilao['codigo'].str.startswith(('WDO','WIN','DOL','IND','CCM')),:]
df3_hora_leilao

,nome,sobrenome,conta,data,hora,datahora,codigo,quantidade,preco,lado,corretagem,bruto,taxas,liquido,duracao,valor_por_contrato,ranking,valor_por_acao
87777,Pedro,Augusto Inácio,NF35,2023-04-03,15:57:00,2023-04-03 15:57:00,RANI3,300.0,8.720000,Venda,0.131,21.000000,0.588,19.555000,3849,0.065183,1,0.007475
87780,Edilson,Jesus dos Santos Junior,NF47,2023-04-03,15:57:05,2023-04-03 15:57:05,RANI3,1800.0,8.746667,Compra,0.787,48.000006,3.542,39.354006,5,0.021863,1,0.002500
87781,Luckas,Rhuan de Oliveira Ruiz Zanforlim,NF54,2023-04-03,16:00:56,2023-04-03 16:00:56,RANI3,800.0,8.710000,Compra,0.348,-48.000000,1.568,-51.845000,234,-0.064806,-1,-0.007440
87782,Eduardo Vinicius,Santos Massoni,NF50,2023-04-03,16:02:00,2023-04-03 16:02:00,RANI3,2500.0,8.710000,Compra,1.089,-25.000000,4.900,-36.985000,300,-0.014794,-1,-0.001699
87806,Felipe,Barcelos Vasconcellos,NF52,2023-04-03,16:15:23,2023-04-03 16:15:23,ENGI11,300.0,39.930000,Compra,0.599,117.000000,2.696,110.444000,1291,0.368147,1,0.009220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91869,Pedro,Augusto Inácio,NF35,2023-04-14,17:07:59,2023-04-14 17:07:59,ABEV3,700.0,14.650000,Venda,0.513,-25.000000,2.308,-30.634000,3664,-0.043763,-1,-0.002987
91873,Luckas,Rhuan de Oliveira Ruiz Zanforlim,NF54,2023-04-14,17:12:00,2023-04-14 17:12:00,GGPS3,200.0,12.640000,Venda,0.126,12.000000,0.568,10.607000,4446,0.053035,1,0.004196
91875,Henrique,de França Santos,2013,2023-04-14,17:13:17,2023-04-14 17:13:17,SAPR11,700.0,19.050000,Compra,0.667,-67.000000,3.000,-74.353000,5292,-0.106219,-1,-0.005576
91876,Luckas,Rhuan de Oliveira Ruiz Zanforlim,NF54,2023-04-14,17:13:18,2023-04-14 17:13:18,SAPR11,100.0,19.050000,Compra,0.095,-11.000000,0.428,-12.051000,3674,-0.120510,-1,-0.006326


####1.4.1 Total Sala

In [72]:
df3_total_leilao = df3_hora_leilao.loc[:,['liquido','codigo']].groupby('codigo').sum().reset_index().sort_values(by='liquido',ascending=False)
df3_total_leilao['liquido'] = round(df3_total_leilao['liquido'],2)
df3_total_leilao

,codigo,liquido
44,MOVI3,925.21
3,ALPA4,504.83
18,CSMG3,414.57
59,SAPR11,389.75
19,CURY3,350.47
...,...,...
27,ENGI11,-177.41
48,ORVR3,-218.37
12,BOAS3,-278.09
37,IVVB11,-314.13


####1.4.2 Op Vencedoras

In [73]:
op_vencedoras_leilao = df3_hora_leilao['liquido'] > 0
df3_vencedoras_leilao = df3_hora_leilao.loc[op_vencedoras_leilao,['codigo','liquido']].groupby('codigo').agg({'liquido': ['mean','count','max']}).reset_index()
df3_vencedoras_leilao.columns = ['codigo','media_melhores','qtd_melhores','melhor']
df3_vencedoras_leilao['media_melhores'] = round(df3_vencedoras_leilao['media_melhores'],2)
df3_vencedoras_leilao['melhor'] = round(df3_vencedoras_leilao['melhor'],2)
df3_vencedoras_leilao

,codigo,media_melhores,qtd_melhores,melhor
0,AGRO3,4.91,1,4.91
1,ALPA4,66.74,9,222.06
2,ALSO3,9.96,1,9.96
3,ALUP11,21.30,3,24.24
4,ASAI3,101.18,4,292.53
...,...,...,...,...
62,TUPY3,66.15,3,152.86
63,UGPA3,37.52,6,59.31
64,UNIP6,41.54,7,82.58
65,VIVA3,20.57,3,33.32


####1.4.3 Op Perdedoras

In [74]:
op_perdedoras_leilao = df3_hora_leilao['liquido'] < 0
df3_perdedoras_leilao = df3_hora_leilao.loc[op_perdedoras_leilao,['codigo','liquido']].groupby('codigo').agg({'liquido': ['mean','count','max']}).reset_index()
df3_perdedoras_leilao.columns = ['codigo','media_piores','qtd_piores','pior']
df3_perdedoras_leilao['media_piores'] = round(df3_perdedoras_leilao['media_piores'],2)
df3_perdedoras_leilao['pior'] = round(df3_perdedoras_leilao['pior'],2)
df3_perdedoras_leilao

,codigo,media_piores,qtd_piores,pior
0,ABCB4,-13.46,1,-13.46
1,ABEV3,-30.63,1,-30.63
2,AGRO3,-18.27,2,-1.44
3,ALPA4,-31.93,3,-27.42
4,ALSO3,-41.95,3,-24.14
5,ALUP11,-25.35,4,-9.17
6,AMBP3,-13.18,3,-3.22
7,ASAI3,-106.85,4,-43.04
8,AZUL4,-186.68,1,-186.68
9,BBAS3,-19.46,1,-19.46


####1.4.4 Valor por Ação

In [75]:
df3_valor_p_acao_leilao = df3_hora_leilao.loc[:,['codigo','valor_por_acao']].groupby('codigo').mean().reset_index().sort_values(by='valor_por_acao',ascending=False)
df3_valor_p_acao_leilao['valor_por_acao'] = df3_valor_p_acao_leilao['valor_por_acao']*100
df3_valor_p_acao_leilao

,codigo,valor_por_acao
75,VULC3,1.007237
39,JHSF3,0.687327
28,EZTC3,0.633613
64,STBP3,0.606386
3,ALPA4,0.601711
...,...,...
48,ORVR3,-0.296208
1,ABEV3,-0.298723
51,PETR4,-0.387453
12,BOAS3,-0.397799


####1.4.5 Tempo Operação

In [76]:
df3_tempo_p_op_leilao = df3_hora_leilao.loc[:,['codigo','duracao']].groupby('codigo').mean().reset_index().sort_values(by='duracao',ascending=False)
df3_tempo_p_op_leilao['duracao'] = round(df3_tempo_p_op_leilao['duracao'],2)
df3_tempo_p_op_leilao

,codigo,duracao
13,BPAC11,7081.00
54,POSI3,5333.00
59,SAPR11,5325.87
34,HASH11,5265.00
70,TUPY3,5157.00
...,...,...
23,ELET3,1113.00
39,JHSF3,1056.00
55,RANI3,890.65
75,VULC3,33.00


####1.4.6 Tabela Final

In [ ]:
df3_combinado = pd.merge(df3_total_leilao,df3_vencedoras_leilao, on='codigo')
df3_combinado1 = pd.merge(df3_combinado, df3_perdedoras_leilao, on='codigo')
df3_combinado2 = pd.merge(df3_combinado1,df3_valor_p_acao_leilao, on='codigo')
df3_combinado3 = pd.merge(df3_combinado2,df3_tempo_p_op_leilao, on='codigo' )
nova_ordem_colunas = ['codigo','liquido','media_melhores','media_piores','melhor','pior','valor_por_acao','qtd_melhores','qtd_piores','duracao']
df3_combinado3.reindex(columns=nova_ordem_colunas)
df3_combinado3['% vencedoras'] = round(df3_combinado3['qtd_melhores']/(df3_combinado3['qtd_melhores'] + df3_combinado3['qtd_piores']),2) 
df3_combinado3

###1.5 X Salada

In [103]:
hora_xsalada = df3_mes['hora'] > '17:00:00'
ativo_futuros = df3_mes['codigo'].str.startswith(('WDO','WIN','DOL','IND')) 
df3_xsalada = df3_mes.loc[hora_xsalada & ativo_futuros,:]
df3_xsalada = df3_xsalada.copy()
df3_xsalada.loc[:, 'codigo'] = df3_xsalada['codigo'].apply(lambda x: x[:3])

In [104]:
df3_xsalada

,nome,sobrenome,conta,data,hora,datahora,codigo,quantidade,preco,lado,corretagem,bruto,taxas,liquido,duracao,valor_por_contrato,ranking,valor_por_acao
87845,Felipe,Massaity Gerez,NF61,2023-04-03,17:46:38,2023-04-03 17:46:38,WDO,4.0,5091.5,Compra,0.20,40.0,3.28,33.04,76,8.26,1,0.001622
87847,Arielton,Silva,NF15,2023-04-03,17:49:51,2023-04-03 17:49:51,WDO,15.0,5088.5,Compra,0.75,75.0,12.30,48.90,7,3.26,1,0.000641
87849,Luckas,Rhuan de Oliveira Ruiz Zanforlim,NF54,2023-04-03,17:51:57,2023-04-03 17:51:57,WIN,6.0,101830.0,Compra,0.30,30.0,1.50,26.40,12,4.40,1,0.000043
87854,Arielton,Silva,NF15,2023-04-03,17:52:37,2023-04-03 17:52:37,WIN,10.0,101795.0,Compra,0.50,60.0,2.50,54.00,9,5.40,1,0.000053
87858,Mateus,Neves,NF43,2023-04-03,17:54:43,2023-04-03 17:54:43,WIN,25.0,101780.0,Compra,1.25,50.0,6.25,35.00,5,1.40,1,0.000014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91975,Arielton,Silva,NF15,2023-04-14,17:57:29,2023-04-14 17:57:29,WDO,10.0,4920.5,Compra,0.50,100.0,8.20,82.60,137,8.26,1,0.001679
91977,Felipe,Barcelos Vasconcellos,NF52,2023-04-14,17:58:30,2023-04-14 17:58:30,WDO,15.0,4920.0,Compra,0.75,150.0,12.30,123.90,30,8.26,1,0.001679
91981,Felipe,Barcelos Vasconcellos,NF52,2023-04-14,17:59:36,2023-04-14 17:59:36,WDO,15.0,4918.0,Compra,0.75,-75.0,12.30,-101.10,10,-6.74,-1,-0.001370
91982,Edilson,Jesus dos Santos Junior,NF47,2023-04-14,17:59:42,2023-04-14 17:59:42,WDO,15.0,4918.6,Compra,0.75,-60.0,12.30,-86.10,23,-5.74,-1,-0.001167


####1.5.1 Total Sala

In [105]:
df3_xsalada_total = df3_xsalada.loc[:,['liquido','codigo']].groupby('codigo').sum().reset_index()
df3_xsalada_total

,codigo,liquido
0,IND,227.900000
1,WDO,1710.540001
2,WIN,517.000000


####1.5.2 Op Vencedoras

In [107]:
op_vencedoras_xsalada = df3_xsalada['liquido'] > 0
df3_vencedoras_xsalada = df3_xsalada.loc[op_vencedoras_xsalada,['codigo','liquido']].groupby('codigo').agg({'liquido': ['mean','count','max']}).reset_index()
df3_vencedoras_xsalada.columns = ['codigo','media_melhores','qtd_melhores','melhor']
df3_vencedoras_xsalada

,codigo,media_melhores,qtd_melhores,melhor
0,IND,138.225000,2,192.15
1,WDO,74.786172,99,298.35
2,WIN,30.228086,108,112.80


####1.5.3 Op Perdedoras

In [109]:
op_perdedoras_xsalada = df3_xsalada['liquido'] < 0
df3_perdedoras_xsalada = df3_xsalada.loc[op_perdedoras_xsalada,['codigo','liquido']].groupby('codigo').agg({'liquido': ['mean','count','min']}).reset_index()
df3_perdedoras_xsalada.columns = ['codigo','media_piores','qtd_piores','pior']
df3_perdedoras_xsalada

,codigo,media_piores,qtd_piores,pior
0,IND,-48.550000,1,-48.55
1,WDO,-123.767197,46,-476.10
2,WIN,-50.882099,54,-281.00


####1.5.4 Valor Por Contrato

In [110]:
df3_valor_p_contrato_xsalada = df3_xsalada.loc[:,['codigo','valor_por_contrato']].groupby('codigo').mean().reset_index()
df3_valor_p_contrato_xsalada

,codigo,valor_por_contrato
0,IND,15.193333
1,WDO,1.311808
2,WIN,0.153762


####1.5.5 Tempo Operação

In [111]:
df3_tempo_op_xsalada = df3_xsalada.loc[:,['codigo','duracao']].groupby('codigo').mean().reset_index()
df3_tempo_op_xsalada

,codigo,duracao
0,IND,165.000000
1,WDO,28.772414
2,WIN,38.339506


####1.5.6 Tabela Final

In [112]:
df3_combinado = pd.merge(df3_xsalada_total,df3_vencedoras_xsalada, on='codigo')
df3_combinado1 = pd.merge(df3_combinado, df3_perdedoras_xsalada, on='codigo')
df3_combinado2 = pd.merge(df3_combinado1,df3_valor_p_contrato_xsalada, on='codigo')
df3_combinado3 = pd.merge(df3_combinado2,df3_tempo_op_xsalada, on='codigo' )
nova_ordem_colunas = ['codigo','resultado','media_melhores','media_piores','melhor','pior','valor_por_contrato','qtd_melhores','qtd_piores','duracao']
df3_combinado3.reindex(columns=nova_ordem_colunas)
#Tabela Final
df3_combinado3['% vencedoras'] = round(df3_combinado3['qtd_melhores']/(df3_combinado3['qtd_melhores'] + df3_combinado3['qtd_piores']),2) 
df3_combinado3

,codigo,liquido,media_melhores,qtd_melhores,melhor,media_piores,qtd_piores,pior,valor_por_contrato,duracao,% vencedoras
0,IND,227.900000,138.225000,2,192.15,-48.550000,1,-48.55,15.193333,165.000000,0.67
1,WDO,1710.540001,74.786172,99,298.35,-123.767197,46,-476.10,1.311808,28.772414,0.68
2,WIN,517.000000,30.228086,108,112.80,-50.882099,54,-281.00,0.153762,38.339506,0.67
